In [2]:
import pandas as pd
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import time
import random

In [ ]:
# random article to strart with
url = "https://news.google.com/rss/articles/CBMiImh0dHBzOi8vd3d3LmZ1dy5jaC9hcnRpY2xlL2FsdHdlZ2fSAQA?oc=5&hl=en-US&gl=US&ceid=US:en"

In [ ]:
email = "insert email here"

In [ ]:
password = "insert password here"

In [ ]:
df = pd.read_excel("fuw_v7.xlsx") # read dataframe with the links to the articles

In [6]:
### Open the browser and go to the website and login

options = webdriver.ChromeOptions()


driver = webdriver.Chrome(options=options)
driver.get(url)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc LQeN7 XWZjwc"]'))).click()  # google cookies
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))).click() # Accept Cookies
driver.maximize_window()
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="NavigationUser_loginbutton__6tRT_"]'))).click() # Login section
input_box = driver.find_element(By.XPATH, '//*[@class="sc-bmzYkS fYzbum"]')  # replace 'input_box' with the actual ID of the input box
input_box.clear()
input_box.send_keys(email)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="first-step-continue-btn"]'))).click()
time.sleep(1)
input_box2 = driver.find_element(By.XPATH, '//*[@class="sc-bmzYkS fYzbum"]')  # replace 'input_box' with the actual ID of the input box
input_box2.clear()
input_box2.send_keys(password)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="native-login-btn"]'))).click()
#manually login

In [8]:
# start the scraping loop
for i in tqdm(range(len(df))):
    if df["Text"][i] == "pending":
        try: 
            driver.execute_script("window.open('about:blank', '_blank');") # open a new tab
            driver.switch_to.window(driver.window_handles[1]) # switch to the new tab

            driver.get(df["URL"][i]) # open the new url

            # Switch back to the first tab
            driver.switch_to.window(driver.window_handles[0])

            # Close the first tab
            driver.close()

            # Switch back to the second tab (which is now the only tab)
            driver.switch_to.window(driver.window_handles[0])

            # Wait until some text is present in the element
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "HtmlText_root__A1OSq")))

            time.sleep(1)

            #find and store the text
            elements = driver.find_elements(By.CLASS_NAME, "HtmlText_root__A1OSq")  # get all elements with the class name "HtmlText_root__A1OSq"
            texts = " \n".join([element.text for element in elements])
            df.loc[i, "Text"] = texts
            time.sleep(1/10)        
        except:
            pass

#driver.quit()



  0%|          | 0/6829 [00:00<?, ?it/s]

100%|██████████| 6829/6829 [00:42<00:00, 162.28it/s]


In [9]:
df.to_excel("fuw_v7.xlsx", index=False)